## Statistics on the unprocessed MIMIC Dataset

In [ ]:
import polars as pl
import os
from datetime import datetime, timedelta
from tabulate import tabulate
import matplotlib.pyplot as plt
import numpy as np

from file_paths import MIMIC_DIR

In [ ]:
TABLE_DIR = "/home/niclas/Dokumente/thesis/results/tables"

In [ ]:
def save_table(table, filename):
    df_pd = table.to_pandas()
    latex_table = tabulate(df_pd, headers='keys', tablefmt='latex')

    with open(os.path.join(TABLE_DIR, filename), "w") as f:
        f.write(latex_table)

Analyze Admissions

In [ ]:
import polars as pl
import os

# Load your data
df = pl.read_csv(os.path.join(MIMIC_DIR, "hosp/admissions.csv"))

# Define a complete mapping to normalize race categories
race_mapping = {
    "HISPANIC/LATINO - MEXICAN": "HISPANIC/LATINO",
    "HISPANIC/LATINO - CENTRAL AMERICAN": "HISPANIC/LATINO",
    "AMERICAN INDIAN/ALASKA NATIVE": "OTHER",
    "SOUTH AMERICAN": "HISPANIC/LATINO",
    "HISPANIC/LATINO - CUBAN": "HISPANIC/LATINO",
    "WHITE - RUSSIAN": "WHITE",
    "WHITE - EASTERN EUROPEAN": "WHITE",
    "PORTUGUESE": "OTHER",  # Often categorized under White due to European origin
    "PATIENT DECLINED TO ANSWER": "UNKNOWN",
    "WHITE - BRAZILIAN": "WHITE",  # Often categorized under White or Hispanic depending on context
    "HISPANIC/LATINO - PUERTO RICAN": "HISPANIC/LATINO",
    "HISPANIC/LATINO - GUATEMALAN": "HISPANIC/LATINO",
    "WHITE": "WHITE",
    "HISPANIC/LATINO - SALVADORAN": "HISPANIC/LATINO",
    "UNKNOWN": "UNKNOWN",
    "HISPANIC/LATINO - COLUMBIAN": "HISPANIC/LATINO",
    "ASIAN - CHINESE": "ASIAN",
    "BLACK/AFRICAN": "BLACK",
    "WHITE - OTHER EUROPEAN": "WHITE",
    "ASIAN - KOREAN": "ASIAN",
    "HISPANIC OR LATINO": "HISPANIC/LATINO",
    "BLACK/CAPE VERDEAN": "BLACK",
    "HISPANIC/LATINO - DOMINICAN": "HISPANIC/LATINO",
    "NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER": "OTHER",
    "ASIAN": "ASIAN",
    "HISPANIC/LATINO - HONDURAN": "HISPANIC/LATINO",
    "ASIAN - ASIAN INDIAN": "ASIAN",
    "BLACK/CARIBBEAN ISLAND": "BLACK",
    "UNABLE TO OBTAIN": "UNKNOWN",
    "BLACK/AFRICAN AMERICAN": "BLACK",
    "OTHER": "OTHER",
    "ASIAN - SOUTH EAST ASIAN": "ASIAN",
    "MULTIPLE RACE/ETHNICITY": "OTHER",
}

# Normalize race column using mapping
df = df.with_columns(
    pl.col("race").map_dict(race_mapping).alias("normalized_race")
)

# Step 1: Remove 'OTHER' and 'UNKNOWN' where possible
df_filtered = df.filter(~pl.col("normalized_race").is_in(["OTHER", "UNKNOWN"]))

# Step 2: Check for remaining subject_ids without valid race after filtering
missing_race_ids = df.join(df_filtered, on="subject_id", how="anti")['subject_id'].unique()

# Step 3: Combine the filtered DataFrame with original for missing subject_ids
df_combined = pl.concat([
    df_filtered,
    df.filter(pl.col("subject_id").is_in(missing_race_ids))
])

# Step 4: Resolve conflicts by taking the first available race
df_aggregated = df_combined.groupby("subject_id").agg(
    pl.col("normalized_race").first().alias("final_race")
)

race_counts = df_aggregated.groupby("final_race").count().rename({'count': 'race_count'})

# Step 6: Calculate percentages
total_patients = df_aggregated.height
race_counts = race_counts.with_columns(
    (pl.col("race_count") / total_patients * 100).alias("percentage")
)

# Convert to a DataFrame for easier viewing (if needed)
race_counts

In [ ]:
import polars as pl
import os

# Load your data
df = pl.read_csv(os.path.join(MIMIC_DIR, "hosp/admissions.csv"))

# Define a complete mapping to normalize race categories
race_mapping = {
    "HISPANIC/LATINO - MEXICAN": "HISPANIC/LATINO",
    "HISPANIC/LATINO - CENTRAL AMERICAN": "HISPANIC/LATINO",
    "AMERICAN INDIAN/ALASKA NATIVE": "OTHER",
    "SOUTH AMERICAN": "HISPANIC/LATINO",
    "HISPANIC/LATINO - CUBAN": "HISPANIC/LATINO",
    "WHITE - RUSSIAN": "WHITE",
    "WHITE - EASTERN EUROPEAN": "WHITE",
    "PORTUGUESE": "OTHER",  # Often categorized under White due to European origin
    "PATIENT DECLINED TO ANSWER": "UNKNOWN",
    "WHITE - BRAZILIAN": "WHITE",  # Often categorized under White or Hispanic depending on context
    "HISPANIC/LATINO - PUERTO RICAN": "HISPANIC/LATINO",
    "HISPANIC/LATINO - GUATEMALAN": "HISPANIC/LATINO",
    "WHITE": "WHITE",
    "HISPANIC/LATINO - SALVADORAN": "HISPANIC/LATINO",
    "UNKNOWN": "UNKNOWN",
    "HISPANIC/LATINO - COLUMBIAN": "HISPANIC/LATINO",
    "ASIAN - CHINESE": "ASIAN",
    "BLACK/AFRICAN": "BLACK",
    "WHITE - OTHER EUROPEAN": "WHITE",
    "ASIAN - KOREAN": "ASIAN",
    "HISPANIC OR LATINO": "HISPANIC/LATINO",
    "BLACK/CAPE VERDEAN": "BLACK",
    "HISPANIC/LATINO - DOMINICAN": "HISPANIC/LATINO",
    "NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER": "OTHER",
    "ASIAN": "ASIAN",
    "HISPANIC/LATINO - HONDURAN": "HISPANIC/LATINO",
    "ASIAN - ASIAN INDIAN": "ASIAN",
    "BLACK/CARIBBEAN ISLAND": "BLACK",
    "UNABLE TO OBTAIN": "UNKNOWN",
    "BLACK/AFRICAN AMERICAN": "BLACK",
    "OTHER": "OTHER",
    "ASIAN - SOUTH EAST ASIAN": "ASIAN",
    "MULTIPLE RACE/ETHNICITY": "OTHER",
}

# Normalize race column using mapping
df = df.with_columns(
    pl.col("race").map_dict(race_mapping).alias("normalized_race")
)

# Define the priority for races (higher number means higher priority)
priority = {
    "UNKNOWN": 0,
    "OTHER": 1,
    "HISPANIC/LATINO": 2,
    "ASIAN": 3,
    "BLACK": 4,
    "WHITE": 5
}

# Add priority column to help with the selection process
df = df.with_columns(
    pl.col("normalized_race").map_dict(priority).alias("priority")
)

# Group by subject_id and resolve conflicts by selecting race with highest priority
# In case of tie, we pick one race at random
df_unique_race = df.groupby("subject_id").agg([
    pl.col("normalized_race").sort_by("priority", descending=True).first(),
])

# Provide an overview of the normalized race distribution
race_distribution = df_unique_race.groupby("normalized_race").agg([
    pl.count().alias("count")
])

race_distribution

In [ ]:
df["subject_id"].unique().len()

In [ ]:
df = pl.read_csv(os.path.join(MIMIC_DIR, "hosp/admissions.csv"))
# Calculate mortality per admission
df = df.with_columns(
    pl.col("hospital_expire_flag").cast(pl.Boolean)
)

# Mortality per admission
mortality_per_admission = df.groupby("hospital_expire_flag").agg([
    pl.count().alias("count")
])

print(mortality_per_admission)

# Mortality per patient
# Group by subject_id and check if the patient has died in any admission
mortality_per_patient = df.groupby("subject_id").agg([
    (pl.col("hospital_expire_flag").max()).alias("died")
])

# Calculate mortality statistics per patient
mortality_per_patient_stats = mortality_per_patient.groupby("died").agg([
    pl.count().alias("count")
])
mortality_per_patient_stats

In [ ]:
# Load the admissions data
df = pl.read_csv(os.path.join(MIMIC_DIR, 'hosp/admissions.csv'))

# Convert the 'admittime' and 'dischtime' columns to datetime
df = df.with_columns([
    pl.col('admittime').str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S').alias('admittime_dt'),
    pl.col('dischtime').str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S').alias('dischtime_dt')
])

# Create the 'overnight' column
df = df.with_columns([
    (pl.col('dischtime_dt') > (pl.col('admittime_dt') + pl.duration(days=1))).alias('overnight')
])

# Group by 'admission_type' and aggregate the 'overnight' column
agg_df = df.groupby('admission_type').agg([
    pl.count('overnight').alias('total_admissions'),
    pl.sum('overnight').alias('overnight_admissions'),
    (pl.sum('overnight') / pl.count('overnight')).alias('overnight_ratio')
])

agg_df_pd = agg_df.to_pandas()
latex_table = tabulate(agg_df_pd, headers='keys', tablefmt='latex')
print(latex_table)

with open(os.path.join(TABLE_DIR, "overnight_admissions_by_adm_type.tex"), "w") as f:
    f.write(latex_table)

Main diagnoses

In [ ]:
diagnoses_df = pl.read_csv(os.path.join(MIMIC_DIR, 'hosp/diagnoses_icd.csv'))
icd_diagnoses_df = pl.read_csv(os.path.join(MIMIC_DIR, 'hosp/d_icd_diagnoses.csv'), schema_overrides={'icd_code': pl.Utf8})


# Filter the data to include only main diagnoses (seq_num == 1)
main_diagnoses_df = diagnoses_df.filter(pl.col('seq_num') == 1)

joined_df = main_diagnoses_df.join(icd_diagnoses_df, on=['icd_code', 'icd_version'], how='left')


# Count the occurrences of each icd_code for main diagnoses
main_diagnoses_count = joined_df.groupby(['icd_version', 'icd_code', 'long_title']).agg([
    pl.count('icd_code').alias('count')
])

# Sort the diagnoses by count in descending order
main_diagnoses_count = main_diagnoses_count.sort('count', descending=True)

# Show the resulting DataFrame
top_20_diagnoses = main_diagnoses_count[0:20]

save_table(top_20_diagnoses, "top_20_main_diagnoses.tex")

All diagnoses

In [ ]:
diagnoses_df = pl.read_csv(os.path.join(MIMIC_DIR, 'hosp/diagnoses_icd.csv'))
icd_diagnoses_df = pl.read_csv(os.path.join(MIMIC_DIR, 'hosp/d_icd_diagnoses.csv'), schema_overrides={'icd_code': pl.Utf8})


joined_df = diagnoses_df.join(icd_diagnoses_df, on=['icd_code', 'icd_version'], how='left')


# Count the occurrences of each icd_code for main diagnoses
diagnoses_count = joined_df.groupby(['icd_version', 'icd_code', 'long_title']).agg([
    pl.count('icd_code').alias('count')
])

# Sort the diagnoses by count in descending order
diagnoses_count = diagnoses_count.sort('count', descending=True)

# Show the resulting DataFrame
top_20_diagnoses = diagnoses_count[0:20]
print(top_20_diagnoses)

save_table(top_20_diagnoses, "top_20_diagnoses.tex")

In [ ]:
# Calculate the number of diagnoses per patient
diagnoses_per_patient = diagnoses_df.groupby('subject_id').agg([
    pl.count('icd_code').alias('num_diagnoses')
])

diagnoses_stats = diagnoses_per_patient["num_diagnoses"].describe()

# Print statistics
print(diagnoses_stats)

In [ ]:
diagnoses_per_patient_pd = diagnoses_per_patient.to_pandas()

# Plot histogram of the number of diagnoses per patient
plt.figure(figsize=(10, 6))
plt.hist(diagnoses_per_patient_pd['num_diagnoses'], bins=30, edgecolor='black', alpha=0.7)
plt.title('Distribution of Diagnoses per Patient')
plt.xlabel('Number of Diagnoses')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# Plot boxplot of the number of diagnoses per patient
plt.figure(figsize=(10, 6))
plt.boxplot(diagnoses_per_patient_pd['num_diagnoses'], vert=False)
plt.title('Boxplot of Diagnoses per Patient')
plt.xlabel('Number of Diagnoses')
plt.grid(True)
plt.show()

# Plot cumulative distribution function (CDF)
sorted_diagnoses = np.sort(diagnoses_per_patient_pd['num_diagnoses'])
yvals = np.arange(1, len(sorted_diagnoses)+1) / float(len(sorted_diagnoses))

plt.figure(figsize=(10, 6))
plt.plot(sorted_diagnoses, yvals)
plt.title('Cumulative Distribution Function of Diagnoses per Patient')
plt.xlabel('Number of Diagnoses')
plt.ylabel('CDF')
plt.grid(True)
plt.show()

In [ ]:
# Load the procedures data
procedures_df = pl.read_csv(os.path.join(MIMIC_DIR, 'hosp/procedures_icd.csv'))

# Load the d_icd_procedures data with icd_code as string
icd_procedures_df = pl.read_csv(os.path.join(MIMIC_DIR, 'hosp/d_icd_procedures.csv'), schema_overrides={'icd_code': pl.Utf8})

# Join the procedures data with the d_icd_procedures data on icd_code and icd_version
joined_procedures_df = procedures_df.join(icd_procedures_df, on=['icd_code', 'icd_version'], how='left')

# Calculate the number of procedures per patient
procedures_per_patient = joined_procedures_df.groupby('subject_id').agg([
    pl.count('icd_code').alias('num_procedures')
])

# Compute statistics
procedures_stats = procedures_per_patient["num_procedures"].describe()

# Print statistics
print(procedures_stats)
save_table(procedures_stats, "procedure_stats.tex")

# Identify and display the top ten most common procedures
top_procedures = joined_procedures_df.groupby(['icd_code', 'long_title']).agg([
    pl.count('icd_code').alias('count')
]).sort('count', descending=True).limit(10)

print(top_procedures)
save_table(top_procedures, "top_10_procedures.tex")

# Convert to Pandas DataFrame for compatibility with plotting libraries
procedures_per_patient_pd = procedures_per_patient.to_pandas()

# Plot histogram of the number of procedures per patient
plt.figure(figsize=(10, 6))
plt.hist(procedures_per_patient_pd['num_procedures'], bins=30, edgecolor='black', alpha=0.7)
plt.title('Distribution of Procedures per Patient')
plt.xlabel('Number of Procedures')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# Plot boxplot of the number of procedures per patient
plt.figure(figsize=(10, 6))
plt.boxplot(procedures_per_patient_pd['num_procedures'], vert=False)
plt.title('Boxplot of Procedures per Patient')
plt.xlabel('Number of Procedures')
plt.grid(True)
plt.show()

# Plot cumulative distribution function (CDF)
sorted_procedures = np.sort(procedures_per_patient_pd['num_procedures'])
yvals = np.arange(1, len(sorted_procedures)+1) / float(len(sorted_procedures))

plt.figure(figsize=(10, 6))
plt.plot(sorted_procedures, yvals)
plt.title('Cumulative Distribution Function of Procedures per Patient')
plt.xlabel('Number of Procedures')
plt.ylabel('CDF')
plt.grid(True)
plt.show()

In [ ]:
# Specify schema overrides for columns that should be treated as strings
schema_overrides = {
    'gsn': pl.Utf8,
    'ndc': pl.Utf8
}

# Load the prescriptions data
prescriptions_df = pl.read_csv(os.path.join(MIMIC_DIR, 'hosp/prescriptions.csv'), schema_overrides=schema_overrides)


# Calculate the number of prescriptions per patient
prescriptions_per_patient = prescriptions_df.groupby('subject_id').agg([
    pl.count('drug').alias('num_prescriptions')
])

# Compute statistics
prescriptions_stats = prescriptions_per_patient["num_prescriptions"].describe()

# Print statistics
print(prescriptions_stats)
save_table(prescriptions_stats, "prescription_stats.tex")

# Identify and display the top ten most common prescriptions
top_prescriptions = prescriptions_df.groupby('drug').agg([
    pl.count('drug').alias('count')
]).sort('count', descending=True).limit(10)

print(top_prescriptions)
save_table(top_prescriptions, "top_10_prescriptions.tex")

# Convert to Pandas DataFrame for compatibility with plotting libraries
prescriptions_per_patient_pd = prescriptions_per_patient.to_pandas()

# Plot histogram of the number of prescriptions per patient
plt.figure(figsize=(10, 6))
plt.hist(prescriptions_per_patient_pd['num_prescriptions'], bins=30, edgecolor='black', alpha=0.7)
plt.title('Distribution of Prescriptions per Patient')
plt.xlabel('Number of Prescriptions')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# Plot boxplot of the number of prescriptions per patient
plt.figure(figsize=(10, 6))
plt.boxplot(prescriptions_per_patient_pd['num_prescriptions'], vert=False)
plt.title('Boxplot of Prescriptions per Patient')
plt.xlabel('Number of Prescriptions')
plt.grid(True)
plt.show()

# Plot cumulative distribution function (CDF)
sorted_prescriptions = np.sort(prescriptions_per_patient_pd['num_prescriptions'])
yvals = np.arange(1, len(sorted_prescriptions)+1) / float(len(sorted_prescriptions))

plt.figure(figsize=(10, 6))
plt.plot(sorted_prescriptions, yvals)
plt.title('Cumulative Distribution Function of Prescriptions per Patient')
plt.xlabel('Number of Prescriptions')
plt.ylabel('CDF')
plt.grid(True)
plt.show()

In [ ]:
admissions_df = pl.read_csv(os.path.join(MIMIC_DIR, 'hosp/admissions.csv'))


# Calculate length of stay
admissions_df = admissions_df.with_columns([
    (pl.col('dischtime').str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S') - pl.col('admittime').str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S')).alias('length_of_stay')
])

# Convert length of stay to days
admissions_df = admissions_df.with_columns([
    (pl.col('length_of_stay').cast(pl.Duration('ns')).dt.hours() / 24).alias('length_of_stay_days')
])

# Compute statistics
los_stats = admissions_df['length_of_stay_days'].describe()

# Print statistics
print(los_stats)

# Percentage of stays longer than 7 days
long_stays_percentage = (admissions_df.filter(pl.col('length_of_stay_days') > 7).height / admissions_df.height) * 100
print(f'Percentage of stays longer than 7 days: {long_stays_percentage:.2f}%')

# Convert to Pandas DataFrame for compatibility with plotting libraries
admissions_df_pd = admissions_df.to_pandas()

# Plot histogram of length of stay
plt.figure(figsize=(10, 6))
plt.hist(admissions_df_pd['length_of_stay_days'], bins=30, edgecolor='black', alpha=0.7)
plt.title('Distribution of Length of Stay')
plt.xlabel('Length of Stay (days)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# Plot boxplot of length of stay
plt.figure(figsize=(10, 6))
plt.boxplot(admissions_df_pd['length_of_stay_days'], vert=False)
plt.title('Boxplot of Length of Stay')
plt.xlabel('Length of Stay (days)')
plt.grid(True)
plt.show()

In [ ]:
mortality_by_diagnosis

In [ ]:
# Calculate overall mortality rate
overall_mortality_rate = (admissions_df['hospital_expire_flag'].sum() / admissions_df.height) * 100
print(f'Overall mortality rate: {overall_mortality_rate:.2f}%')

# Join with diagnoses data to analyze mortality by specific diagnoses
main_diagnoses_df = diagnoses_df.filter(pl.col('seq_num') == 1)

# Join admissions with main diagnoses
admissions_with_diagnoses = admissions_df.join(main_diagnoses_df, on='hadm_id', how='left')

# Calculate mortality rate by diagnosis
mortality_by_diagnosis = admissions_with_diagnoses.groupby(['icd_code', 'icd_version']).agg([
    pl.count('icd_code').alias('total_admissions'),
    pl.sum('hospital_expire_flag').alias('total_deaths')
])

mortality_by_diagnosis = mortality_by_diagnosis.with_columns([
    (pl.col('total_deaths') / pl.col('total_admissions') * 100).alias('mortality_rate')
])

# Join with ICD descriptions
mortality_by_diagnosis = mortality_by_diagnosis.join(icd_diagnoses_df, on=['icd_code', 'icd_version'], how='left')

# Print top diagnoses by mortality rate
top_mortality_diagnoses = mortality_by_diagnosis.sort('mortality_rate', descending=True).limit(10)
print(top_mortality_diagnoses)
